In [ ]:
import random
import math
import pandas as pd


num_flights = 20  # Total number of flights (Airbus 320 + Boeing 738)
num_classA_pilots = 25
num_classB_pilots = 25
num_classA_copilots = 25
num_classB_copilots = 25
num_cabin_crew = 100

# Generate random flight schedule data and fixed operation costs
flights = [f"Flight-{i+1}" for i in range(num_flights)]
boeing_flights = random.sample(flights, random.randint(5, 20))
for flight in boeing_flights:
    flights.remove(flight)
    flights.append(f"Boeing-{flight.split('-')[1]}")
random.shuffle(flights)

operation_costs = {flight: random.randint(5000, 10000) for flight in flights}

cost_per_pilot = 2000
cost_per_copilot = 1500
cost_per_cabin_crew = 1000

#initial solution
def generate_initial_solution():
    crew_assignments = {}
    for flight in flights:
        num_pilots_assigned = 1
        num_copilots_assigned = random.randint(1, 2)
        num_cabin_crew_assigned = random.randint(1, 5)

        if 'Boeing' in flight: #class B
            pilots = random.sample(range(num_classA_pilots + 1, num_classA_pilots + num_classB_pilots + 1), num_pilots_assigned)
            copilots = random.sample(range(num_classA_copilots + 1, num_classA_copilots + num_classB_copilots + 1), num_copilots_assigned)
        else:  #Class A
            pilots = random.sample(range(1, num_classA_pilots + 1), num_pilots_assigned)
            copilots = random.sample(range(1, num_classA_copilots + 1), num_copilots_assigned)

        cabin_crew = random.sample(range(1, num_cabin_crew + 1), num_cabin_crew_assigned)

        crew_assignments[flight] = {'Pilots': pilots, 'CoPilots': copilots, 'CabinCrew': cabin_crew}
    return crew_assignments

# Cost Function
def calculate_cost(crew_assignments):
    total_cost = 0
    for flight, assigned_crews in crew_assignments.items():
        flight_cost = operation_costs[flight]

        pilot_cost = len(assigned_crews['Pilots']) * cost_per_pilot
        copilot_cost = len(assigned_crews['CoPilots']) * cost_per_copilot
        cabin_crew_cost = len(assigned_crews['CabinCrew']) * cost_per_cabin_crew

        total_cost += flight_cost + pilot_cost + copilot_cost + cabin_crew_cost
    return total_cost

# Simulated Annealing function
def simulated_annealing(initial_solution, calculate_cost, temperature):
    current_solution = initial_solution
    best_solution = initial_solution
    current_cost = calculate_cost(current_solution)
    best_cost = current_cost

    while temperature > 0.1:
        for flight in flights:
            current_crews = current_solution[flight]

            current_crews['Pilots'] = random.sample(range(1, num_classA_pilots + 1), 1) if 'Airbus' in flight else random.sample(range(num_classA_pilots + 1, num_classA_pilots + num_classB_pilots + 1), 1)
            current_crews['CoPilots'] = random.sample(range(1, num_classA_copilots + 1), random.randint(1, 2)) if 'Airbus' in flight else random.sample(range(num_classA_copilots + 1, num_classA_copilots + num_classB_copilots + 1), random.randint(1, 2))
            current_crews['CabinCrew'] = random.sample(range(1, num_cabin_crew + 1), random.randint(1, 5))

        new_cost = calculate_cost(current_solution)

        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
            current_cost = new_cost
            best_solution = current_solution

        temperature *= 0.95  # cooling rate

    return best_solution, current_cost


results = []
num_iterations = 200

for i in range(num_iterations):
    initial_solution = generate_initial_solution()
    optimal_solution, optimal_cost = simulated_annealing(initial_solution, calculate_cost, 1000)
    results.append({'Iteration': i + 1, 'Optimal Cost': optimal_cost})

results_df = pd.DataFrame(results)

results_df.to_excel('optimal_costs_C1.xlsx', index=False)
